# Estudo de Parcelamento das Dívidas

Nesse notebook pretendo programar um código que recebe uma tabela CSV e então realiza o calculo dos valores das parcelas por mês até finalizar a dívida.
Como forma de estudar a quantidade de meses que vai ser necessário para finaliza-la.

## Importações

In [1]:
import pandas as pd
from datetime import date

## Funções Auxiliares

In [2]:
def to_month_start(dt: pd.Timestamp) -> pd.Timestamp:
    """
        Normaliza uma data para o 1º dia do mês.
    """
    return pd.Timestamp(year=dt.year, month=dt.month, day=1)

In [3]:
def parse_brl_number(x):
    """
        Converte '1.287,36' ou '1287,36' ou '1287.36' em float.
        Aceita também valores numéricos.
    """
    if pd.isna(x):
        return 0.0
    if isinstance(x, (int, float)):
        return float(x)
    s = str(x).strip()
    # remove "R$" e espaços
    s = s.replace("R$", "").replace(" ", "")
    # remove separador de milhar '.' e troca ',' por '.'
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return 0.0

In [4]:
def build_monthly_schedule(df: pd.DataFrame, start_mode: str = "current_month") -> pd.DataFrame:
    """
        Gera tabela mensal com quanto será pago em cada mês.
        
        start_mode:
          - "current_month": começa sempre no mês atual (como você pediu)
          - "start_date": começa na data de início da dívida (quando ela existe)
    """
    # Ajuste dos nomes esperados conforme seu CSV
    col_name = "Nome da Dívida"
    col_total_parc = "Nº de Parcelas"
    col_parc_value = "Valor da Parcela"
    col_paid = "Parcelas Pagas"
    col_start = "Data de Início"
    col_status = "Status da Dívida"

    # Garantir colunas
    missing = [c for c in [col_name, col_total_parc, col_parc_value, col_paid] if c not in df.columns]
    if missing:
        raise ValueError(f"Colunas faltando no CSV: {missing}")

    # Filtra só ativas, se existir coluna de status
    if col_status in df.columns:
        df = df[df[col_status].astype(str).str.strip().str.lower() == "ativa"].copy()

    # Converte números
    df[col_total_parc] = pd.to_numeric(df[col_total_parc], errors="coerce").fillna(0).astype(int)
    df[col_paid] = pd.to_numeric(df[col_paid], errors="coerce").fillna(0).astype(int)
    df[col_parc_value] = df[col_parc_value].apply(parse_brl_number)

    # Parcelas restantes
    df["parcelas_restantes"] = (df[col_total_parc] - df[col_paid]).clip(lower=0)

    # Define mês inicial do cronograma
    today = pd.Timestamp(date.today())
    current_month = to_month_start(today)

    # Parse de data de início (pode ter "(Indefinido)")
    if col_start in df.columns:
        start_dates = pd.to_datetime(df[col_start], format="%d/%m/%Y", errors="coerce")
    else:
        start_dates = pd.Series([pd.NaT] * len(df), index=df.index)

    rows = []
    for idx, row in df.iterrows():
        n = int(row["parcelas_restantes"])
        if n <= 0:
            continue

        parcela = float(row[col_parc_value])
        nome = str(row[col_name])

        if start_mode == "start_date" and pd.notna(start_dates.loc[idx]):
            start_month = to_month_start(start_dates.loc[idx])
            # próxima parcela após as já pagas
            start_month = start_month + pd.DateOffset(months=int(row[col_paid]))
            # se quiser forçar não começar antes do mês atual, descomente:
            # start_month = max(start_month, current_month)
        else:
            # modo pedido: sempre a partir do mês atual
            start_month = current_month

        # Gera n meses de pagamentos
        for m in range(n):
            pay_month = start_month + pd.DateOffset(months=m)
            rows.append(
                {
                    "mes": pay_month.strftime("%Y-%m"),
                    "nome_divida": nome,
                    "valor_pago_no_mes": parcela,
                }
            )

    if not rows:
        return pd.DataFrame(columns=["mes", "total_pago_no_mes"])

    sched = pd.DataFrame(rows)

    # Soma total por mês
    monthly = (
        sched.groupby("mes", as_index=False)["valor_pago_no_mes"]
        .sum()
        .rename(columns={"valor_pago_no_mes": "total_pago_no_mes"})
        .sort_values("mes")
    )

    return monthly

In [9]:
def main(
    input_csv_path: str,
    output_csv_path: str = "pagamento_mensal.csv",
    start_mode: str = "current_month",
):
    # Lê CSV (separador ; e vírgula decimal)
    df = pd.read_csv(input_csv_path, sep=";", dtype=str, keep_default_na=False)

    monthly = build_monthly_schedule(df, start_mode=start_mode)

    # Salva com separador ; e decimal , (pt-BR)
    monthly.to_csv(output_csv_path, sep=";", index=False, float_format="%.2f")
    print(f"OK! Gerado: {output_csv_path}")
    print(monthly.head(12))

    # Retorna
    return monthly

## Execução da Operação

In [10]:
%%time
# Exemplo de uso:
# main("dividas.csv", "pagamento_mensal.csv", start_mode="current_month")

df = main("dividas.csv", "pagamento_mensal.csv", start_mode="current_month")

OK! Gerado: pagamento_mensal.csv
        mes  total_pago_no_mes
0   2026-02             654.49
1   2026-03             654.49
2   2026-04             563.90
3   2026-05             431.40
4   2026-06             431.40
5   2026-07             431.40
6   2026-08             379.41
7   2026-09             303.34
8   2026-10             256.35
9   2026-11              53.64
10  2026-12              53.64
11  2027-01              53.64
CPU times: total: 15.6 ms
Wall time: 9.1 ms


In [11]:
df

,mes,total_pago_no_mes
0,2026-02,654.49
1,2026-03,654.49
2,2026-04,563.90
3,2026-05,431.40
4,2026-06,431.40
5,2026-07,431.40
6,2026-08,379.41
7,2026-09,303.34
8,2026-10,256.35
9,2026-11,53.64
